In [1]:
from tqdm import tqdm
import pandas as pd
import os
import gzip
import zipfile
import shutil
from time import sleep
import pymorphy2
from pyaspeller import YandexSpeller
from lru import LRU
import pickle
from time import sleep
from concurrent.futures import ThreadPoolExecutor
import tarfile
import bz2
import sys
import numpy as np
with open('ill_url_ids.pickle', 'rb') as file:
    ill_url_ids = pickle.load(file)
def transform(arg, ill_url_ids):
    if arg not in ill_url_ids:
        return arg
    else:
        return ill_url_ids[arg]
alpha = np.array([0.41,0.16,0.105,0.08,0.06,0.05,0.35,0.3,0.25,0.2])
class URL_info():
    def __init__(self):
        self.total_shows = 0
        self.total_clicks = 0
        self.num_first_clicks = 0
        self.num_last_clicks = 0
        self.show_poses = np.zeros((10,),dtype='int')
        self.show_poses_cl = np.zeros((10,),dtype='int')
        self.click_poses = np.zeros((10,),dtype='int')
        self.time = 0
        self.clicks_on_higher_pos = 0
        self.clicks_on_lower_pos = 0
        self.clicks_after_this = 0
        self.clicks_before_this = 0
        self.is_click_on_higher_pos = 0
        self.is_click_on_lower_pos = 0

In [2]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [3]:
all_groups

,QueryId,DocumentId
0,0,1443
1,0,5912
2,0,5963
3,0,6096
4,0,6230
...,...,...
606045,6310,497350
606046,6310,512263
606047,6310,529857
606048,6310,551291


In [4]:
vals = all_groups['DocumentId'].values
qifs = all_groups['QueryId'].values

In [5]:
with open("doc_to_host.pickle", 'rb') as file:
    doc_to_host = pickle.load(file)
with open("QDP.pickle", 'rb') as file:
    url_simple = pickle.load(file)
with open("QHP.pickle", 'rb') as file:
    host_simple = pickle.load(file)

In [6]:
template = np.arange(1,11)
click_feats = dict()
click_feats1 = dict()

for i in range(2,11):
    click_feats['QD_cumul_ctr{}'.format(i)] = []
for i in range(2,11):
    click_feats1['QH_cumul_ctr{}'.format(i)] = []

click_feats['QD_clicks_out'] = []
click_feats['QD_shows_out'] = []
click_feats['QD_clicks_out_prob'] = []
click_feats['QD_shows_out_prob'] = []

click_feats1['QH_clicks_out'] = []
click_feats1['QH_shows_out'] = []
click_feats1['QH_clicks_out_prob'] = []
click_feats1['QH_shows_out_prob'] = []

click_feats['QD_higher_prob'] = []
click_feats['QD_lower_prob'] = []
click_feats['QD_num_higher_cl'] = []
click_feats['QD_num_lower_cl'] = []
click_feats1['QH_higher_prob'] = []
click_feats1['QH_lower_prob'] = []
click_feats1['QH_num_higher_cl'] = []
click_feats1['QH_num_lower_cl'] = []

In [7]:
for i in tqdm(range(len(vals))):
    key = vals[i]
    Q = qifs[i]
    key1 = transform(key, ill_url_ids)
    
    for j in range(2,11):
        var = np.sum(url_simple[Q][key1].click_poses[:j])/(np.sum(url_simple[Q][key1].show_poses[:j])+0.0000001)
        click_feats['QD_cumul_ctr{}'.format(j)].append(var)
        
    click_feats['QD_clicks_out'].append(url_simple[Q][key1].total_clicks - np.sum(url_simple[Q][key1].show_poses_cl))
    click_feats['QD_shows_out'].append(url_simple[Q][key1].total_shows - np.sum(url_simple[Q][key1].show_poses))
    var = url_simple[Q][key1].total_clicks - np.sum(url_simple[Q][key1].show_poses_cl)
    click_feats['QD_clicks_out_prob'].append(var/(url_simple[Q][key1].total_clicks+0.0000001))
    var = url_simple[Q][key1].total_shows - np.sum(url_simple[Q][key1].show_poses)
    click_feats['QD_shows_out_prob'].append(var/(url_simple[Q][key1].total_shows+0.0000001))
    
    click_feats['QD_higher_prob'].append(url_simple[Q][key1].is_click_on_higher_pos/(url_simple[Q][key1].total_shows+0.0000001))
    click_feats['QD_lower_prob'].append(url_simple[Q][key1].is_click_on_lower_pos/(url_simple[Q][key1].total_shows+0.0000001))
    click_feats['QD_num_higher_cl'].append(url_simple[Q][key1].is_click_on_higher_pos)
    click_feats['QD_num_lower_cl'].append(url_simple[Q][key1].is_click_on_lower_pos)

100%|██████████| 606050/606050 [01:39<00:00, 6108.59it/s]


In [8]:
for i in tqdm(range(len(vals))):
    key = vals[i]
    Q = qifs[i]
    key1 = transform(key, ill_url_ids)
    key1 = doc_to_host[key1]
    
    for j in range(2,11):
        var = np.sum(host_simple[Q][key1].click_poses[:j])/(np.sum(host_simple[Q][key1].show_poses[:j])+0.0000001)
        click_feats1['QH_cumul_ctr{}'.format(j)].append(var)
        
    click_feats1['QH_clicks_out'].append(host_simple[Q][key1].total_clicks - np.sum(host_simple[Q][key1].show_poses_cl))
    click_feats1['QH_shows_out'].append(host_simple[Q][key1].total_shows - np.sum(host_simple[Q][key1].show_poses))
    var = host_simple[Q][key1].total_clicks - np.sum(host_simple[Q][key1].show_poses_cl)
    click_feats1['QH_clicks_out_prob'].append(var/(host_simple[Q][key1].total_clicks+0.0000001))
    var = host_simple[Q][key1].total_shows - np.sum(host_simple[Q][key1].show_poses)
    click_feats1['QH_shows_out_prob'].append(var/(host_simple[Q][key1].total_shows+0.0000001))
    
    click_feats1['QH_higher_prob'].append(host_simple[Q][key1].is_click_on_higher_pos/(host_simple[Q][key1].total_shows+0.0000001))
    click_feats1['QH_lower_prob'].append(host_simple[Q][key1].is_click_on_lower_pos/(host_simple[Q][key1].total_shows+0.0000001))
    click_feats1['QH_num_higher_cl'].append(host_simple[Q][key1].is_click_on_higher_pos)
    click_feats1['QH_num_lower_cl'].append(host_simple[Q][key1].is_click_on_lower_pos)

100%|██████████| 606050/606050 [01:41<00:00, 5970.22it/s]


In [9]:
with open("click_feats_pair_extra.pickle", 'wb') as file:
    pickle.dump(click_feats, file)

In [10]:
with open("click_feats_pair_extra1.pickle", 'wb') as file:
    pickle.dump(click_feats1, file)